# **WELCOME TO THIS NOTEBOOK**

Let's install pyspark

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
!pip install pyspark

Importing the modules

In [80]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [81]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

# Loading the dataset

In [82]:
file_path ='/content/drive/MyDrive/dataset/listenings.csv'
df_listenings =spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path)
df_listenings
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Cleaning tables

In [83]:
df_listenings = df_listenings.drop('date')
df_listenings = df_listenings.na.drop()
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [84]:
row_numbers =df_listenings.count()
columns_numbers =len(df_listenings.columns)


In [85]:
print(columns_numbers,row_numbers)

4 13758905



# Let's Perform some aggregation
to see how many times each user has listened to specific track


In [86]:
df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id','track').agg(count('*')).alias('count').orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+--------+
|user_id|               track|count(1)|
+-------+--------------------+--------+
| --Seph| White Winter Hymnal|       3|
| --Seph|         The Funeral|       1|
| --Seph|Hope There's Someone|       1|
| --Seph|         The Painter|       1|
| --Seph|          Je te veux|       1|
| --Seph|            War Pigs|       1|
| --Seph|                 F12|       1|
| --Seph|                Team|       1|
| --Seph|          Nightmares|       1|
| --Seph|               Radio|       1|
| --Seph|   All I Want Is You|       1|
| --Seph|    Little by Little|       2|
| --Seph|        After Nature|       1|
| --Seph|In the Hall of th...|       1|
| --Seph|   Hey There Delilah|       1|
| --Seph|   Let's Call It Off|       1|
| --Seph|               Leloo|       1|
| --Seph|             Pack Up|       1|
| --Seph|           Introitus|       1|
| --Seph|        The Leanover|       1|
+-------+--------------------+--------+
only showing top 20 rows



In [87]:
row_numbers = df_listenings_agg.count()
columns_numbers = len(df_listenings_agg.columns)
print(row_numbers,columns_numbers)

9930128 3


In [88]:
df_listenings_agg = df_listenings_agg.limit(20000)

# Let's convert the user id and track columns into unique integers




In [89]:
indexer = [StringIndexer(inputCol=col, outputCol=col+'_index', handleInvalid='keep').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns) - set(['count']))]


In [90]:
pipeline = Pipeline(stages=indexer)
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()

+-------+--------------------+--------+--------------+-------------+-----------+
|user_id|               track|count(1)|count(1)_index|user_id_index|track_index|
+-------+--------------------+--------+--------------+-------------+-----------+
| --Seph|          Nightmares|       1|           0.0|         69.0|    10600.0|
| --Seph|Virus (Luke Fair ...|       1|           0.0|         69.0|    15893.0|
| --Seph|Airplanes [feat H...|       1|           0.0|         69.0|      521.0|
| --Seph|Belina (Original ...|       1|           0.0|         69.0|     3280.0|
| --Seph|              Monday|       1|           0.0|         69.0|      334.0|
| --Seph|Hungarian Dance No 5|       1|           0.0|         69.0|     7555.0|
| --Seph|       Life On Mars?|       1|           0.0|         69.0|     1164.0|
| --Seph|  California Waiting|       1|           0.0|         69.0|      195.0|
| --Seph|       Phantom Pt II|       1|           0.0|         69.0|     1378.0|
| --Seph|   Summa for String

In [91]:
data = data.select('user_id_index' , 'track_index', 'count(1)').orderBy('user_id_index')


# Train and Test data

In [92]:
(training , test) =data.randomSplit([0.5,0.5])

# Let's Create our Model

In [93]:
USERID ='user_id_index'
TRACK ='track_index'
COUNT ='count(1)'

als = ALS(maxIter=5, regParam=0.01, userCol=USERID, itemCol=TRACK, ratingCol=COUNT)
model = als.fit(training)
predictions = model.transform(test)



# Generate top 10 Track recommendations for each user

In [94]:
recs = model.recommendForAllUsers(10)

In [95]:
recs.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{16908, 15.18897...|
|            1|[{4460, 19.753273...|
|            2|[{4460, 32.794292...|
|            3|[{16908, 12.90741...|
|            4|[{304, 13.077294}...|
|            5|[{13561, 6.854038...|
|            6|[{4460, 14.372103...|
|            7|[{304, 9.573789},...|
|            8|[{957, 3.7499871}...|
|            9|[{2474, 4.962798}...|
|           10|[{4460, 32.386124...|
|           11|[{4460, 13.843725...|
|           12|[{4460, 26.586012...|
|           13|[{16908, 9.100163...|
|           14|[{16908, 19.69618...|
|           15|[{13561, 9.109789...|
|           16|[{175, 6.343837},...|
|           17|[{13561, 7.888008...|
|           18|[{4460, 22.538296...|
|           19|[{233, 8.825232},...|
+-------------+--------------------+
only showing top 20 rows



In [96]:
recs.take(1)

[Row(user_id_index=0, recommendations=[Row(track_index=16908, rating=15.188973426818848), Row(track_index=128, rating=7.910904884338379), Row(track_index=1689, rating=7.069347858428955), Row(track_index=13583, rating=4.979283809661865), Row(track_index=5041, rating=4.979283809661865), Row(track_index=478, rating=4.882370471954346), Row(track_index=8389, rating=4.517340183258057), Row(track_index=406, rating=4.439211368560791), Row(track_index=15476, rating=4.231089115142822), Row(track_index=14052, rating=4.106672763824463)])]